In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from fastai import *
from fastai.tabular import *
from fastai.callbacks import *
from fastai.text import *
from fastai.data_block import *
from fastai.metrics import *
from sklearn.model_selection import StratifiedKFold
from IPython.display import FileLink
from sklearn.model_selection import *

In [3]:
model_name='short_description-60k'
# model_name='full-60k'
# model_name='full-30k'
# model_name='samlple-30k'

In [4]:
path=Path('data')

In [ ]:
df_raw = pd.read_parquet(path/'processed/training_stage_4.parquet', engine='fastparquet')

In [6]:
df_root_cause_counts = pd.DataFrame(df_raw.groupby(['root_cause']).size().sort_values(ascending=False), columns=['count'])
min_size = 30
large_cats = df_root_cause_counts[df_root_cause_counts["count"] > min_size]
total_count = int(df_root_cause_counts.sum())
total_big_enough_covered =float(large_cats.sum()*100/df_root_cause_counts.sum())
print(f'Total cats: {len(df_root_cause_counts)} with {total_count} items')
print(f'cats w/ >{min_size} items: {len(large_cats)} with {total_big_enough_covered:.2f}% coverage')

Total cats: 960 with 1156151 items
cats w/ >30 items: 276 with 99.67% coverage


In [7]:
df_train = df_raw[df_raw['lang'] == 'en'][['short_description', 'root_cause']]
train_data = df_train[df_train['root_cause'].isin(large_cats.reset_index()['root_cause'])]

In [10]:
split = train_test_split(range(len(train_data)), random_state=42, test_size=0.035, stratify=train_data['root_cause'])

In [ ]:
##### Packaging for models

In [12]:
# bs=64
# data_lm = load_data(path, f'data-lm-{model_name}.pkl', bs=bs)
# data_clas = (TextList.from_df(train_data, path, vocab=data_lm.vocab)
#              .split_by_idxs(split[0], split[1])
#              .label_from_df('root_cause')
#              .databunch(bs=bs))
# data_clas.save(f'data_clas_{model_name}.pkl')

In [5]:
bs = 128 # 192 breaks in unfreeze(); 256 breaks in freeze(-3)
data_clas = load_data(path, f'data_clas_{model_name}.pkl', bs=bs)

### Classifier

In [12]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.0)
learn.load_encoder(f'{model_name}-tuned-enc')
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3), moms=(0.8, 0.7))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(1e-4/(2.6**4),1e-4), moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,2.122345,2.075880,0.410555,02:19


epoch,train_loss,valid_loss,accuracy,time
0,2.063885,2.006952,0.423227,02:40


epoch,train_loss,valid_loss,accuracy,time
0,2.049393,1.998162,0.425999,03:33


In [17]:
learn.unfreeze()
callbacks = SaveModelCallback(learn, every='improvement', mode='max', monitor='accuracy', name=f'nlp-classifier-{model_name}-final')
learn.fit_one_cycle(3, slice(1e-4/(2.6**4),1e-4), moms=(0.8, 0.7), callbacks=callbacks)

epoch,train_loss,valid_loss,accuracy,time
0,1.993981,1.989168,0.428243,04:46
1,2.013729,1.978530,0.430461,05:09
2,1.981924,1.978127,0.430699,05:42


Better model found at epoch 0 with accuracy value: 0.4282433092594147.
Better model found at epoch 1 with accuracy value: 0.43046095967292786.
Better model found at epoch 2 with accuracy value: 0.43069854378700256.


epoch,train_loss,valid_loss,accuracy,time
0,1.986600,1.975588,0.430540,06:08
